<a href="https://colab.research.google.com/github/uol-mediaprocessing-202021/medienverarbeitung-e-interactive-camera-system/blob/main/Documentation/Documentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>